In [1]:
import pandas as pd
import numpy as np
import re
import datetime
from dateutil.relativedelta import relativedelta
from datetime import date
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
%matplotlib inline

# Importing Data set

In [16]:
file = r'C:\Users\Mr. Jarvis\Desktop\python project\project 1\Consumer_Complaints_train.csv'

In [17]:
dada_train = pd.read_csv(file)

In [18]:
file1 = r'C:\Users\Mr. Jarvis\Desktop\python project\project 1\Consumer_Complaints_test_share.csv'

In [19]:
data_test = pd.read_csv(file1)

In [20]:
data_test['Consumer disputed?']=np.nan

In [22]:
data_train['data'] = 'train'
data_test['data'] = 'test'

TypeError: 'str' object does not support item assignment

In [14]:
data_test = data_test[data_train.columns]

AttributeError: 'str' object has no attribute 'columns'

In [15]:
data_all=pd.concat([data_train,data_test],axis=0)

TypeError: cannot concatenate object of type '<class 'str'>'; only Series and DataFrame objs are valid

# Removing Garbage columns

In [6]:
garbage_cols=data_all.isnull().sum()[data_all.isnull().sum()>(0.5*data_all.shape[0])].keys() 

In [7]:
data_all.drop(columns=garbage_cols,axis=1,inplace=True)

In [8]:
data_all.drop(columns=["Complaint ID"],inplace=True)

# Imputing Missing Values

In [9]:
data_all["Sub-product"]=np.where(data_all["Sub-product"].isnull(),"not_available",data_all["Sub-product"])

In [10]:
data_all["State"]=np.where(data_all["State"].isnull(),"not_available",data_all["State"])

In [11]:
data_all["ZIP code"]=np.where(data_all["ZIP code"].isnull(),"not_available",data_all["ZIP code"])

In [12]:
data_all["Submitted via"]=np.where(data_all["Submitted via"].isnull(),data_all["Submitted via"].mode()[0],data_all["Submitted via"])

# Formatting date

In [13]:
d1=pd.to_datetime(data_all["Date received"])
d2=pd.to_datetime(data_all["Date sent to company"])
data_all["resolution_gap_days"]=(d2-d1)/np.timedelta64(1,'D')
data_all.drop(columns =["Date received","Date sent to company"], inplace = True)

# Creating Dummies

In [14]:
def dummies(data,var,freq_cutoff=0):
    t= data[var].value_counts(normalize=True) 
    t=t[t.values>freq_cutoff] 
    t=t.sort_values() 
    t_min=t.idxmin()
    t=t.drop([t_min])
    categories=t.index

    for cat in categories :
        name=var+'_'+cat
        name=re.sub(" ","",name) 
        name=re.sub("-","_",name)
        name=re.sub("\\?","Q",name) 
        name=re.sub("<","LT_",name)
        name=re.sub("\\+","",name) 
        name=re.sub("\\/","_",name) 
        name=re.sub(">","GT_",name) 
        name=re.sub("=","EQ_",name)
        name=re.sub(",","",name)
        data[name]=(data[var]==cat)+0 
               
    data=data.drop(columns=[var])
    return data

In [15]:
data_all=dummies(data_all,"Product",0.05)
data_all=dummies(data_all,"Sub-product",0.05)
data_all=dummies(data_all,"Issue",0.05)
data_all=dummies(data_all,"Company",0.05)
data_all=dummies(data_all,"State",0.05)
data_all=dummies(data_all,"ZIP code",0.05)
data_all=dummies(data_all,"Submitted via",0.05)
data_all=dummies(data_all,"Company response to consumer",0.05)
data_all=dummies(data_all,"Timely response?")

# splitting back in train and test 

In [16]:
data_train=data_all[data_all['data']=='train'] 
del data_train['data'] 
data_test=data_all[data_all['data']=='test']
data_test.drop(['Consumer disputed?','data'],axis=1,inplace=True) 

C:\Users\jayes\Anaconda3\lib\site-packages\pandas\core\frame.py:4102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


# splitting train in train1 and train2 

In [17]:
train1, train2 = train_test_split(data_train, test_size = 0.25,random_state=2)

x_train1=train1.drop(["Consumer disputed?"],1)
y_train1=train1["Consumer disputed?"]

x_train2=train2.drop(["Consumer disputed?"],1)
y_train2=train2["Consumer disputed?"]

x_train1.reset_index(drop=True,inplace=True)
y_train1.reset_index(drop=True,inplace=True)

In [18]:
import warnings 
warnings.filterwarnings('ignore')

model=LogisticRegression(fit_intercept=True)
params={'class_weight':['balanced'],
        'n_jobs':[-1],
'penalty':['l1','l2'],
'C':np.linspace(0.1,1,1000,10)}
grid_search=GridSearchCV(model,param_grid=params,cv=5,scoring="roc_auc")

In [ ]:
grid_search.fit(x_train1,y_train1) # grid search fits a Logistic Regression mode

In [ ]:
logr=grid_search.best_estimator_

In [ ]:
logr.fit(x_train1,y_train1)

In [ ]:
train1_score=logr.predict_proba(x_train1)[:,1]
train1_score

In [ ]:
train2_score=logr.predict_proba(x_train2)[:,1]
train2_score

In [ ]:
train1_classes=(train1_score>0.5).astype(int)
y_train1_classes=np.where(y_train1=="Yes",1,0)
train1_score[train1_score>0.5].size*100/train1_score.size #percent of Yes in train 1

In [ ]:
train2_classes=(train2_score>0.5).astype(int)
y_train2_classes=np.where(y_train2=="Yes",1,0)
train2_score[train2_score>0.5].size*100/train2_score.size #percent of Yes in train 2

In [ ]:
roc_auc_score(y_train1, train1_classes) #auc score of train 1

In [ ]:
roc_auc_score(y_train2, train2_classes) #auc score of train 2